In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, FIRST_COMPLETED, wait
import numpy as np
import tritonclient.grpc as grpcclient
from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

In [ ]:
render = False
model_name = "facenet_ensemble"

filenames = index_directory(image_folder)
triton_client = grpcclient.InferenceServerClient(url=url, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=model_name, model_version=model_version
)

model_config = triton_client.get_model_config(
    model_name=model_name, model_version=model_version
).config

input_name, output_metadata, batch_size = parse_model_grpc(model_metadata, model_config)
output_names = [i.name for i in output_metadata]
max_workers = max_batch_size * model_config.instance_group[0].count

In [ ]:
## smoke test
image_wise_bboxes = []
while len(image_wise_bboxes) == 0:
    rand_idx = np.random.randint(0, len(filenames))
    filename = filenames[rand_idx]
    infer_result = submit_to_facedetect(
        filename, input_name, output_names, request_id=filename
    )

    image_wise_bboxes = infer_result.as_numpy(output_names[0]).reshape(-1, 4)

print(image_wise_bboxes)
print("{} {} {}".format(filename, input_name, output_names))
render_image(filename, image_wise_bboxes)

In [ ]:
results = []
errors = []
future_to_request = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures_done = set()
    futures_notdone = set()

    for filename in tqdm(filenames, desc="Image Inference"):
        futures_notdone.add(
            executor.submit(
                submit_to_facedetect, filename, input_name, output_names, filename
            )
        )
        if len(futures_notdone) >= max_workers:
            done, futures_notdone = wait(futures_notdone, return_when=FIRST_COMPLETED)
            futures_done.update(done)

future_to_request = 0

for future in futures_done:
    future_to_request += 1
    result = {}
    try:

        infer_result = future.result()
        this_id = infer_result.get_response().id
        image_wise_bboxes = infer_result.as_numpy(output_names[0]).reshape(-1, 4)
        image_probas = infer_result.as_numpy(output_names[1]).reshape(-1, 1)
        result["filename"] = this_id
        result["faces"] = [
            {"bbox": {"x1": 0, "y1": 0, "x2": 0, "y2": 0}, "probability": 0}
        ]
        for bbox, proba in zip(image_wise_bboxes, image_probas):
            result["faces"] = [
                {
                    "bbox": {
                        "x1": bbox[0],
                        "y1": bbox[1],
                        "x2": bbox[2],
                        "y2": bbox[3],
                    },
                    "probability": proba[0],
                }
                for bbox, proba in zip(image_wise_bboxes, image_probas)
            ]

        results.append(result)
    except Exception as exc:
        print(exc)

with open(json_data_filename, "w") as f:
    json.dump(results, f, default=to_serializable_int)

In [ ]:
if render:
    print("Rendering unique files in filenames...")
    rendered_images = []
    img_w = 800
    for this_id in results:
        image_wise_bboxes = results[this_id][output_names[0]]
        img = render_image(this_id, image_wise_bboxes)
        wpercent = img_w / float(img.size[0])
        hsize = int(float(img.size[1]) * float(wpercent))
        img = img.resize((img_w, hsize), Image.ANTIALIAS)
        img.show()

___